In [1]:
!nvidia-smi

Sat Jan 31 06:25:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/112.6 GB disk)


In [4]:
!pip install roboflow
import roboflow

rf = roboflow.Roboflow(api_key="6caCwRWvb0wM2NJ4xkJk")
project = rf.workspace("parkingspaces-bs0l9").project("parking-space-detection-sp8zb")
version = project.version(1)
dataset = version.download("yolov8", overwrite=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 115.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.13.0.90
    Uninstalling opencv-python-headless-4.13.0.90:
      Successfully uninstalled opencv-python-headless-4.13.0.90
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Parking-Space-Detection-1 in yolov8:: 100%|██████████| 4812/4812 [00:00<00:00, 4869.46it/s]


In [1]:
# Create a dedicated folder in your Drive first
!mkdir -p "/content/drive/MyDrive/Roboflow_Datasets"

# Copy the entire folder to that new Drive location
!cp -r "/content/Parking-Space-Detection-1" "/content/drive/MyDrive/Roboflow_Datasets/"

In [3]:
"""
PARKING SPACE DETECTION MODEL TRAINING SCRIPT - PERSISTENT COLAB VERSION
========================================================================
Author: Mihini Boteju
Date: 31 January 2026
"""

import os
import torch
from ultralytics import YOLO
from google.colab import drive

# ============================================================================
# 1. MOUNT GOOGLE DRIVE & SETUP PERSISTENT PATHS
# ============================================================================
print("Mounting Google Drive...")
drive.mount('/content/drive')

# All training results (best.pt, results.csv, plots) will be saved here
# This folder will stay in your Google Drive even after Colab resets.
DRIVE_RESULTS_DIR = '/content/drive/MyDrive/Parking_Project_Results'
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)

# ============================================================================
# 2. CONFIGURATION
# ============================================================================
MODEL_NAME = 'yolov8n-seg.pt'  # Nano segmentation model (Fastest for Colab)

# Update this to the folder Roboflow created in your local /content directory
# Example: '/content/Parking-Space-1/data.yaml'
try:
    DATASET_YAML = f"{dataset.location}/data.yaml"
except NameError:
    # Manual fallback if 'dataset' object isn't defined from previous cells
    DATASET_YAML = '/content/Parking-Space-Detection-1/data.yaml'

EPOCHS = 100
IMG_SIZE = 640
BATCH_SIZE = 16  # Good for T4 GPU memory
PATIENCE = 50

# ============================================================================
# 3. UTILITY FUNCTIONS
# ============================================================================

def get_best_device():
    """Detect NVIDIA GPU for Colab"""
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✅ Detected: NVIDIA GPU - {gpu_name}")
        return '0'
    else:
        print("⚠️  No GPU detected - Go to Runtime > Change Runtime Type and select T4!")
        return 'cpu'

# ============================================================================
# 4. MAIN TRAINING EXECUTION
# ============================================================================

def train_parking_model():
    print("=" * 70)
    print("🅿️ PARKING SPACE DETECTION: DRIVE-PERSISTENT TRAINING")
    print("=" * 70)

    # Verify Dataset exists before starting
    if not os.path.exists(DATASET_YAML):
        print(f"❌ ERROR: Dataset not found at: {DATASET_YAML}")
        print("Please ensure your Roboflow import or unzip step ran successfully.")
        return False

    # Load Model
    model = YOLO(MODEL_NAME)
    device = get_best_device()

    print(f"\nTraining Results will be saved to: {DRIVE_RESULTS_DIR}")
    print("-" * 70)

    try:
        # START TRAINING
        # The 'project' argument sends everything directly to your Google Drive
        results = model.train(
            data=DATASET_YAML,
            epochs=EPOCHS,
            imgsz=IMG_SIZE,
            batch=BATCH_SIZE,
            device=device,
            patience=PATIENCE,
            save=True,
            project=DRIVE_RESULTS_DIR,
            name='parking_segmentation_run', # This creates a subfolder in your Drive
            exist_ok=True,
            workers=2,
            cache=True,
            plots=True
        )

        print("\n" + "=" * 70)
        print("🎉 TRAINING COMPLETED AND SAVED TO DRIVE!")
        print("=" * 70)

        # Point to the persistent best model
        best_model_path = os.path.join(DRIVE_RESULTS_DIR, 'parking_segmentation_run/weights/best.pt')
        print(f"Your permanent model is at: {best_model_path}")

        return True

    except Exception as e:
        print(f"\n❌ TRAINING FAILED: {e}")
        return False

if __name__ == '__main__':
    train_parking_model()

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🅿️ PARKING SPACE DETECTION: DRIVE-PERSISTENT TRAINING
✅ Detected: NVIDIA GPU - Tesla T4

Training Results will be saved to: /content/drive/MyDrive/Parking_Project_Results
----------------------------------------------------------------------
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Parking-Space-Detection-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchs